# Running the workflow for Robust-Rail-NL

In [ ]:
import subprocess
import json
import time
import pytz 
from datetime import datetime

### TODO
- configurations for scenario generation
    - give service tasks
    - *matching*
- evaluator does not seems to do the move before exit, and should allow move to exit and then exit
- fix problems in solver
    - Move action with start=end and no resources: scenario 6
    - *write 0 values in json plans*
    - *move with earlier end than start time*

### Pipeline for creating scenarios from a configuration file

In [ ]:
scenario_configurations = {
    1:  {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 7200,
        "seed": 101,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 10,
        "min_gap_on_gateway": 400,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3", "SNG-4"],
            "super_type_ratio": 0,
            "units_per_composition": [2]
        },
        "gateway": {
            "arrival": ["41"],
            "departure": ["41"]
        }
    },
    2:   {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 3600,
        "seed": 102,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 10,
        "min_gap_on_gateway": 200,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3", "SNG-4"],
            "super_type_ratio": 0.4,
            "units_per_composition": [1, 2, 3]
        },
        "gateway": {
            "arrival": ["41"],
            "departure": ["41"]
        }
    },
    3:  {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 4800,
        "seed": 103,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 20,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3", "SNG-4"],
            "super_type_ratio": 0.3,
            "units_per_composition": [1, 2, 3]
        },
        "matching": 2,
        "mixed_traffic": False,
        "gateway": {
            "arrival": ["41"],
            "departure": ["41"]
        }
    },
    4:  {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 4800,
        "seed": 104,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 20,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3", "SNG-4"],
            "super_type_ratio": 0.5,
            "units_per_composition": [1, 2, 3]
        },
        "matching": 1,
        "gateway": {
            "arrival": ["41"],
            "departure": ["41"]
        }
    },
    5:   {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 10800,
        "seed": 105,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 20,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3", "SNG-4"],
            "super_type_ratio": 0.5,
            "units_per_composition": [1, 2],
            "instanding_ratio": 0.2,
            "outstanding_ratio": 0.2    
        },
        "gateway": {
            "arrival": ["41"],
            "departure": ["41"]
        },
        "matching": 0,
    }
}

In [ ]:
timeout = 60
for scenario_number, config_content in scenario_configurations.items():
    with open(f"./scenario_config_{scenario_number}.json", 'w') as config_file:
        json.dump(config_content, config_file, indent=4)
    try:
        result = subprocess.run(['python', '/workspace/robust-rail-generator/src/main.py', "-c", f"./scenario_config_{scenario_number}.json", "-p", ".", "-s", f"./scenario_{scenario_number}.json"], timeout=timeout, stdout=subprocess.PIPE)
        if result.returncode != 0:
            print(f"Generator for scenario {scenario_number} failed with return code {result.returncode}.")
    except subprocess.TimeoutExpired:
        print(f"Generator timed out after {timeout} seconds.")

### Pipeline for getting different plans from the solver for a given scenario

In [ ]:
# Method to run the solver for a given plan number. 
# It updates the configuration file with the scenario and plan paths, then executes the solver using the updated configuration in subprocess bounded by the timeout.
def run_solver(plan_number, scenario_number, timeout, seed=42, print_output=False):
    with open("./config_solver.yaml", "r") as f:
        config = f.readlines()
        for i, line in enumerate(config):
            if line.split(": ")[0] == "ScenarioPath":
                config[i] = f'ScenarioPath: "/workspace/scenario-planning-inputs/Scenario_settings/Kleine_Binckhorst_landmarks/scenario_solver_{scenario_number}.json"\n'
            elif line.split(": ")[0] == "PlanPath":
                config[i] = f'PlanPath: "/workspace/scenario-planning-inputs/Scenario_settings/Kleine_Binckhorst_landmarks/scenario_{scenario_number}_plan_{plan_number}.json"\n'
            elif line.split(": ")[0] == "Seed":
                config[i] = f'Seed: {seed}\n'
            elif line.split(": ")[0] == "MaxDuration":
                config[i] = f'MaxDuration: {timeout}\n'
            elif line.split(": ")[0] == "TemporaryPlanPath":
                config[i] = f'TemporaryPlanPath: "/workspace/scenario-planning-inputs/Scenario_settings/Kleine_Binckhorst_landmarks/tmp_plans/scenario_{scenario_number}_plan_{plan_number}"\n'
        with open("./config_solver.yaml", "w") as newf:
            newf.writelines(config)
    try:
        print("*** Now running the solver for plan number:", plan_number, "with seed", seed, "***")
        start_time = time.time()
        result = subprocess.run(['dotnet', 'run', '--project', '/workspace/robust-rail-solver/ServiceSiteScheduling/HIP.csproj', '--', '--config=./config_solver.yaml'], timeout=timeout, stdout=subprocess.PIPE)
        if result.returncode != 0:
            print(f"Solver failed with return code {result.returncode}.")
        else:
            print(f"Solver completed in {time.time() - start_time:.2f} seconds.")
        if print_output:
            print("---------- Output: -----------")
            print(result.stdout.decode())
            print("------------------------------")
    except subprocess.TimeoutExpired:
        print(f"Solver timed out after {timeout} seconds.")

In [ ]:
current_timeout = 600
current_time = str(datetime.now().astimezone(pytz.timezone("Europe/Amsterdam"))).replace(' ', '_').replace(":", "").split(".")[0]
output_file = f"stats/generation_{current_time}.csv"
with open(output_file, "w") as f:
    f.write("scenario,plan,seed,execution_time,timeout\n")
scenarios = [
    # "0",
    # "1",
    "2",
    # "3",
    # "4",
    # "5",
    # "6"
]
for scenario in scenarios:
    print(f"Running scenario {scenario} with timeout {current_timeout} seconds.")
    for plan_number in range(1, 6):
        seed = 10 * int(scenario) + plan_number * 2
        start_time = time.time()
        run_solver(plan_number, scenario, current_timeout, seed, print_output=False)
        run_time = time.time() - start_time
        with open(output_file, "a") as f:
            f.write(f"{scenario},{plan_number},{seed},{run_time},{current_timeout}\n")
        print(f"Finished running plan {plan_number} for scenario {scenario}.")
    print("")